# Create new project

In [5]:
import deeplabcut
import os

objs = ['teabag', 'cup', 'pitcher', 'tap']
project_name = f'make_tea_{obj}'
experimenter = 'luke'
videos = ['/home/luke/Desktop/project/make_tea/camera-main/videos/1641420386/1641420386-left.mp4',
          '/home/luke/Desktop/project/make_tea/camera-main/videos/1641421641/1641421641-right.mp4']
working_directory = os.getcwd()
for obj in objs:
    deeplabcut.create_new_project(project_name, 
                              experimenter, 
                              videos, 
                              working_directory=working_directory, 
                              copy_videos=False, multianimal=True)


Created "/home/luke/Desktop/project/make_tea/dlc/make_tea_teabag-luke-2022-01-29/videos"
Created "/home/luke/Desktop/project/make_tea/dlc/make_tea_teabag-luke-2022-01-29/labeled-data"
Created "/home/luke/Desktop/project/make_tea/dlc/make_tea_teabag-luke-2022-01-29/training-datasets"
Created "/home/luke/Desktop/project/make_tea/dlc/make_tea_teabag-luke-2022-01-29/dlc-models"
Attempting to create a symbolic link of the video ...
Created the symlink of /home/luke/Desktop/project/make_tea/camera-main/videos/1641420386/1641420386-left.mp4 to /home/luke/Desktop/project/make_tea/dlc/make_tea_teabag-luke-2022-01-29/videos/1641420386-left.mp4
Created the symlink of /home/luke/Desktop/project/make_tea/camera-main/videos/1641421641/1641421641-right.mp4 to /home/luke/Desktop/project/make_tea/dlc/make_tea_teabag-luke-2022-01-29/videos/1641421641-right.mp4
/home/luke/Desktop/project/make_tea/dlc/make_tea_teabag-luke-2022-01-29/videos/1641420386-left.mp4
/home/luke/Desktop/project/make_tea/dlc/make_t

Gtk-Message: 12:42:20.626: Failed to load module "atk-bridge"
Gtk-Message: 12:42:20.627: Failed to load module "canberra-gtk-module"


# Modify labeled-data

In [3]:
from glob import glob
import os
import pandas as pd

obj = 'cup'

data_folders = glob(f'/home/luke/Desktop/project/make_tea/dlc/make_tea_{obj}*/labeled-data/*')
for folder in data_folders:
    imgs = glob(folder + '/*.png')
    h5file = glob(folder + '/*.h5')[0]
    df = pd.read_hdf(h5file)
    if obj != 'teabag':
        partial_df = df['luke_yinghan']['single'][[f'{obj}_h',f'{obj}_m',f'{obj}_t']]
        pdindex = pd.MultiIndex. from_product([['luke'], [obj], ['bodypart1', 'bodypart2', 'bodypart3'], ['x', 'y']], names=['scorer', 'individuals','bodyparts', 'coords'])
    else:
        partial_df = df['luke_yinghan'][['individual1', 'individual2']]
        pdindex = pd.MultiIndex. from_product([['luke'], [obj + '1', obj + '2'], ['bodypart1', 'bodypart2'], ['x', 'y']], names=['scorer', 'individuals','bodyparts', 'coords'])
    inds = df.index
    df_new = pd.DataFrame(partial_df.values, columns = pdindex, index = inds)
    project_dir = os.path.dirname(os.path.dirname(folder))
    for ind in inds:
        if project_dir + '/' + ind not in imgs:
            df_new.drop(index = ind, inplace = True)
    outputname = folder + '/CollectedData_luke.h5'
    df_new.to_hdf(outputname, key = 'data')
    df_new.to_csv(outputname.split(".h5")[0] + ".csv")

# Check labels

In [7]:
import deeplabcut

obj = 'teabag'
config_path = glob(f'/home/luke/Desktop/project/make_tea/dlc/make_tea_{obj}*/config.yaml')[0]
deeplabcut.check_labels(config_path)

Creating images with labels by luke.


Gtk-Message: 16:13:40.975: Failed to load module "atk-bridge"
Gtk-Message: 16:13:40.976: Failed to load module "canberra-gtk-module"
100%|███████████████████████████████████████████| 55/55 [01:04<00:00,  1.17s/it]

If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


# Train

In [1]:
obj = 'teabag'
config_path = glob(f'/home/luke/Desktop/project/make_tea/dlc/make_tea_{obj}*/config.yaml')[0]

deeplabcut.create_multianimaltraining_dataset(config_path)
deeplabcut.train_network(config_path, gputouse = 0, saveiters = 10000, displayiters=100, maxiters=50000)


Help on function train_network in module deeplabcut.pose_estimation_tensorflow.training:

train_network(config, shuffle=1, trainingsetindex=0, max_snapshots_to_keep=5, displayiters=None, saveiters=None, maxiters=None, allow_growth=False, gputouse=None, autotune=False, keepdeconvweights=True, modelprefix='')
    Trains the network with the labels in the training dataset.
    
    Parameter
    ----------
    config : string
        Full path of the config.yaml file as a string.
    
    shuffle: int, optional
        Integer value specifying the shuffle index to select for training. Default is set to 1
    
    trainingsetindex: int, optional
        Integer specifying which TrainingsetFraction to use. By default the first (note that TrainingFraction is a list in config.yaml).
    
    Additional parameters:
    
    max_snapshots_to_keep: int, or None. Sets how many snapshots are kept, i.e. states of the trained network. Every savinginteration many times
        a snapshot is stored, h

Gtk-Message: 13:24:00.742: Failed to load module "atk-bridge"
Gtk-Message: 13:24:00.745: Failed to load module "canberra-gtk-module"


# Pick outliers after training

In [1]:
import deeplabcut
from deeplabcut.refine_training_dataset import find_outliers_in_raw_data

config_path = '/home/luke/Desktop/project/make_tea/dlc/make_tea_01052022-luke_yinghan-2022-01-05/config.yaml'

Gtk-Message: 11:59:23.060: Failed to load module "atk-bridge"
Gtk-Message: 11:59:23.062: Failed to load module "canberra-gtk-module"


## Pick the videos(in a list) to check training result

In [19]:
videos = ['/home/luke/Desktop/project/make_tea/camera-main/videos/1641420386/1641420386-right.mp4','/home/luke/Desktop/project/make_tea/camera-main/videos/1641421641/1641421641-left.mp4']

## Analyze videos. 

***_full.pickle and _meta.pickle file are produced***

In [ ]:
import deeplabcut
from glob import glob

obj = 'teabag'
config_path = glob(f'/home/luke/Desktop/project/make_tea/dlc/make_tea_{obj}*/config.yaml')[0]
vid_id = '1642994619'
videos = f'/home/luke/Desktop/project/make_tea/camera-main/videos/{vid_id}/left/{vid_id}-left.mp4'

deeplabcut.analyze_videos(config_path, videos , videotype='.mp4')

Using snapshot-50000 for model /home/luke/Desktop/project/make_tea/dlc/make_tea_teabag-luke-2022-01-29/dlc-models/iteration-0/make_tea_teabagJan29-trainset95shuffle1


/home/luke/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Activating extracting of PAFs


2022-01-31 13:50:14.047483: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-31 13:50:14.051546: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-31 13:50:14.051823: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-31 13:50:14.052145: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

## Check the result in video. The result videos are in the video directories.

In [4]:
deeplabcut.create_video_with_all_detections(config_path, videos, videotype='.mp4')

Creating labeled video for  1642994619-left


100%|█████████████████████████████████████████| 538/538 [00:06<00:00, 87.76it/s]


## If result good, continue to get tracklets. If not good, find outliers and re-train the network

In [11]:
pickle_file = '/home/luke/Desktop/project/make_tea/camera-main/videos/1641421641/1641421641-leftDLC_dlcrnetms5_make_tea_01052022Jan5shuffle1_50000_full.pickle'
find_outliers_in_raw_data(config_path, pickle_file, videos[1])

Frames from video 1641421641-right  already extracted (more will be added)!
Loading video...
Duration of video [s]:  43.5 , recorded @  20.0 fps!
Overall # of frames:  870 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 43.5  seconds.
Let's select frames indices: [23, 38, 39, 40, 41, 68, 69, 70, 71, 72, 73, 76, 78, 79, 80, 91, 93, 97, 100, 109, 115, 123, 124, 128, 135, 136, 151, 155, 167, 191, 231, 236, 237, 239, 241, 242, 243, 244, 247, 269, 275, 283, 286, 314, 316, 365, 366, 381, 382, 407, 439, 614, 615, 616, 617, 618, 623, 624, 657, 687]
Creating the symbolic link of the video
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\1641421641-right.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.


where pickle_file is the _full.pickle one obtains after video analysis. Flagged frames will be added to your collection of images in the corresponding labeled-data folders for you to label. You can choose to re-train the network after labeling the outlier images.

# Supposed retrained and now track the objects

## Convert detections to tracklets. 

***Remember to set track_method to be 'box' because the default 'ellipse' method does not work. _assemblies.pickle and _bx.pickle file will be produced***

In [21]:
deeplabcut.convert_detections2tracklets(config_path, videos[0], videotype= 'mp4', overwrite = True, track_method = 'box')
deeplabcut.convert_detections2tracklets(config_path, videos[1], videotype= 'mp4', overwrite = True, track_method = 'box')

Using snapshot-50000 for model /home/luke/Desktop/project/make_tea/dlc/make_tea_01052022-luke_yinghan-2022-01-05/dlc-models/iteration-0/make_tea_01052022Jan5-trainset95shuffle1
Processing...  /home/luke/Desktop/project/make_tea/camera-main/videos/1641420386/1641420386-right.mp4
/home/luke/Desktop/project/make_tea/camera-main/videos/1641420386  already exists!
Analyzing /home/luke/Desktop/project/make_tea/camera-main/videos/1641420386/1641420386-rightDLC_dlcrnetms5_make_tea_01052022Jan5shuffle1_50000.h5


100%|███████████████████████████████████████| 776/776 [00:00<00:00, 6861.46it/s]
776it [00:00, 831.44it/s] 


The tracklets were created. Now you can 'refine_tracklets'.
Using snapshot-50000 for model /home/luke/Desktop/project/make_tea/dlc/make_tea_01052022-luke_yinghan-2022-01-05/dlc-models/iteration-0/make_tea_01052022Jan5-trainset95shuffle1
Processing...  /home/luke/Desktop/project/make_tea/camera-main/videos/1641421641/1641421641-left.mp4
/home/luke/Desktop/project/make_tea/camera-main/videos/1641421641  already exists!
Analyzing /home/luke/Desktop/project/make_tea/camera-main/videos/1641421641/1641421641-leftDLC_dlcrnetms5_make_tea_01052022Jan5shuffle1_50000.h5


100%|███████████████████████████████████████| 870/870 [00:00<00:00, 8844.35it/s]
870it [00:00, 1938.70it/s]

The tracklets were created. Now you can 'refine_tracklets'.


## Stitch tracklets to get .h5 files

In [40]:
deeplabcut.stitch_tracklets(config_path, videos[0], videotype='mp4',
                            shuffle=1, trainingsetindex=0, track_method = 'box')
deeplabcut.stitch_tracklets(config_path, videos[1], videotype='mp4',
                            shuffle=1, trainingsetindex=0, track_method = 'box')

Processing...  /home/luke/Desktop/project/make_tea/camera-main/videos/1641420386/1641420386-right.mp4
/home/luke/Desktop/project/make_tea/camera-main/videos/1641420386  already exists!


100%|███████████████████████████████████████████| 8/8 [00:00<00:00, 7612.17it/s]


Processing...  /home/luke/Desktop/project/make_tea/camera-main/videos/1641421641/1641421641-left.mp4
/home/luke/Desktop/project/make_tea/camera-main/videos/1641421641  already exists!


100%|███████████████████████████████████████████| 6/6 [00:00<00:00, 6785.07it/s]


## Refine tracklets. Not working for now, need to open up the GUI to do so

In [41]:
track_files =['/home/luke/Desktop/project/make_tea/camera-main/videos/1641420386/1641420386-rightDLC_dlcrnetms5_make_tea_01052022Jan5shuffle1_50000_bx.pickle',
              '/home/luke/Desktop/project/make_tea/camera-main/videos/1641421641/1641421641-leftDLC_dlcrnetms5_make_tea_01052022Jan5shuffle1_50000_bx.pickle']

deeplabcut.refine_tracklets(config_path, track_files[0], videos[0])
deeplabcut.refine_tracklets(config_path, track_files[1], videos[1])

## Filter the tracks

In [46]:
dlc.filterpredictions(config_path, videos[0], track_method = 'box')
dlc.filterpredictions(config_path, videos[1], track_method = 'box')

Filtering with median model /home/luke/Desktop/project/make_tea/camera-main/videos/1641420386/1641420386-right.mp4
Data from 1641420386-right were already filtered. Skipping...
Filtering with median model /home/luke/Desktop/project/make_tea/camera-main/videos/1641421641/1641421641-left.mp4
Saving filtered csv poses!


## Plots

In [50]:
dlc.plot_trajectories(config_path, videos,filtered = True, track_method = 'box')

Loading  /home/luke/Desktop/project/make_tea/camera-main/videos/1641420386/1641420386-right.mp4 and data.
Plots created! Please check the directory "plot-poses" within the video directory


## Create videos

In [51]:
dlc.create_labeled_video(config_path, videos ,filtered = True, track_method = 'box')

/home/luke/Desktop/project/make_tea/camera-main/videos/1641421641/home/luke/Desktop/project/make_tea/camera-main/videos/1641420386   already exists! already exists!

Starting to process video: /home/luke/Desktop/project/make_tea/camera-main/videos/1641420386/1641420386-right.mp4
Loading /home/luke/Desktop/project/make_tea/camera-main/videos/1641420386/1641420386-right.mp4 and data.
Starting to process video: /home/luke/Desktop/project/make_tea/camera-main/videos/1641421641/1641421641-left.mp4
Loading /home/luke/Desktop/project/make_tea/camera-main/videos/1641421641/1641421641-left.mp4 and data.
Duration of video [s]: 38.8, recorded with 20.0 fps!
Overall # of frames: 776 with cropped frame dimensions: 1920 1088
Generating frames and creating video.
Duration of video [s]: 43.5, recorded with 20.0 fps!
Overall # of frames: 870 with cropped frame dimensions: 1920 1088
Generating frames and creating video.


100%|█████████████████████████████████████████| 870/870 [00:11<00:00, 78.84it/s]
